## Transform the data to Star Schema

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType
from pyspark.sql import functions as f
from pyspark.sql.types import StringType

### fact_payment

In [0]:
payments = spark.table("default.stg_payments")
display(payments.limit(4))

payment_id,date_id,amount,rider_id
1,2019-05-01,9.0,1000
2,2019-06-01,9.0,1000
3,2019-07-01,9.0,1000
4,2019-08-01,9.0,1000


In [0]:
spark.sql("drop table if exists default.fact_payment")
payments.dropDuplicates(["payment_id"]).write.format("delta").mode("overwrite").saveAsTable("default.fact_payment")

### dim_rider

In [0]:
riders = spark.table("default.stg_riders")
display(riders.limit(4))

rider_id,first_name,last_name,address,birthday,account_start_date,account_end_date,is_member
1000,Diana,Clark,1200 Alyssa Squares,1989-02-13,2019-04-23,null,True
1001,Jennifer,Smith,397 Diana Ferry,1976-08-10,2019-11-01,2020-09-01,True
1002,Karen,Smith,644 Brittany Row Apt. 097,1998-08-10,2022-02-04,null,True
1003,Bryan,Roberts,996 Dickerson Turnpike,1999-03-29,2019-08-26,null,False


In [0]:
spark.sql("drop table if exists default.dim_rider")
riders.dropDuplicates(["rider_id"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_rider")

### fact_trip

In [0]:
trips = spark.table("default.stg_trips")
display(trips.limit(4))

trip_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,rider_id
89E7AA6C29227EFF,classic_bike,2021-02-12 16:14:56,2021-02-12 16:21:43,525,660,71934
0FEFDE2603568365,classic_bike,2021-02-14 17:52:38,2021-02-14 18:12:09,525,16806,47854
E6159D746B2DBB91,electric_bike,2021-02-09 19:10:18,2021-02-09 19:19:10,KA1503000012,TA1305000029,70870
B32D3199F1C2E75B,classic_bike,2021-02-02 17:49:41,2021-02-02 17:54:06,637,TA1305000034,58974
83E463F23575F4BF,electric_bike,2021-02-23 15:07:23,2021-02-23 15:22:37,13216,TA1309000055,39608
BDAA7E3494E8D545,electric_bike,2021-02-24 15:43:33,2021-02-24 15:49:05,18003,KP1705001026,36267
A772742351171257,classic_bike,2021-02-01 17:47:42,2021-02-01 17:48:33,KP1705001026,KP1705001026,50104
295476889D9B79F8,classic_bike,2021-02-11 18:33:53,2021-02-11 18:35:09,18003,18003,19618
362087194BA4CC9A,classic_bike,2021-02-27 15:13:39,2021-02-27 15:36:36,KP1705001026,KP1705001026,16732
21630F715038CCB0,classic_bike,2021-02-20 08:59:42,2021-02-20 09:17:04,KP1705001026,KP1705001026,57068


In [0]:
fact_trip = trips.join(riders, trips.rider_id == riders.rider_id,"inner") \
            .withColumn('duration_in_min', round((unix_timestamp("ended_at") - unix_timestamp('started_at'))/60)) \
            .withColumn('rider_age', round(datediff(to_date("started_at"), to_date("birthday"))/365.25)) \
            .select("trip_id", riders.rider_id, "start_station_id", "end_station_id", "started_at", "ended_at", "rideable_type","duration_in_min", "rider_age")
            
display(fact_trip.limit(4))

trip_id,rider_id,start_station_id,end_station_id,started_at,ended_at,rideable_type,duration_in_min,rider_age
222BB8E5059252D7,34062,KA1503000064,13021,2021-06-13 09:48:47,2021-06-13 10:07:23,classic_bike,19.0,30.0
1826E16CB5486018,5342,TA1306000010,13021,2021-06-21 22:59:13,2021-06-21 23:04:29,classic_bike,5.0,26.0
3D9B6A0A5330B04D,3714,TA1305000030,13021,2021-06-18 16:06:42,2021-06-18 16:12:02,classic_bike,5.0,26.0
07E82F5E9C9E490F,18793,TA1305000034,13021,2021-06-17 16:46:23,2021-06-17 17:02:45,classic_bike,16.0,19.0
A8E94BAECBF0C2DD,43342,TA1308000009,TA1308000009,2021-06-13 17:36:29,2021-06-13 18:30:39,docked_bike,54.0,28.0
378F4AB323AA1D14,6693,TA1308000009,TA1308000009,2021-06-13 13:20:10,2021-06-13 14:06:14,docked_bike,46.0,29.0
38AD311DC2EB1FBE,71480,KA1503000019,KA1503000019,2021-06-16 17:14:30,2021-06-16 17:28:34,docked_bike,14.0,56.0
1D466737F0B18097,50846,TA1308000009,TA1308000009,2021-06-27 14:51:52,2021-06-27 15:26:39,docked_bike,35.0,41.0
27E1142E1ACFAEFB,18951,13257,13257,2021-06-21 13:58:26,2021-06-21 13:58:53,electric_bike,0.0,22.0
67F2A115DAE77924,63987,TA1308000009,TA1308000009,2021-06-22 00:51:43,2021-06-22 01:08:25,classic_bike,17.0,37.0


In [0]:
spark.sql("drop table if exists default.fact_trip")
fact_trip.write.format("delta").mode("overwrite").saveAsTable("default.fact_trip")

### dim_station

In [0]:
stations = spark.table("default.stg_stations")
display(stations.limit(4))

station_id,name,latitude,longitude
525,Glenwood Ave & Touhy Ave,42.012701,-87.66605799999999
KA1503000012,Clark St & Lake St,41.88579466666667,-87.63110066666668
637,Wood St & Chicago Ave,41.895634,-87.672069
13216,State St & 33rd St,41.8347335,-87.6258275


In [0]:
spark.sql("drop table if exists default.dim_station")
stations.dropDuplicates(["station_id"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_station")

### dim_date

In [0]:
min_date = trips.selectExpr('MIN(started_at) as started_at').first().asDict()['started_at']
max_date = trips.selectExpr('MAX(ended_at) as ended_at').first().asDict() ['ended_at']

expression = f"sequence(to_date('{min_date}'), to_date('{max_date}'), interval 1 day)"
dim_date = spark.createDataFrame([(1,)], ["date_id"])

dim_date = dim_date.withColumn("dateinit", f.explode(f.expr(expression)))
dim_date = dim_date.withColumn("date", f.to_timestamp(dim_date.dateinit, "yyyy-MM-dd"))

dim_date = dim_date \
            .withColumn("day", f.dayofmonth(dim_date.date)) \
            .withColumn("month", f.month(dim_date.date)) \
            .withColumn("quarter", f.quarter(dim_date.date)) \
            .withColumn("year", f.year(dim_date.date)) \
            .withColumn("day_of_year", f.dayofyear(dim_date.date)) \
            .withColumn("day_of_week", f.dayofweek(dim_date.date)) \
            .withColumn("date_id", dim_date.date.cast(StringType())) \
            .drop(f.col("dateinit"))

display(dim_date.limit(4))

date_id,date,day,month,quarter,year,day_of_year,day_of_week
2021-02-01 00:00:00,2021-02-01T00:00:00Z,1,2,1,2021,32,2
2021-02-02 00:00:00,2021-02-02T00:00:00Z,2,2,1,2021,33,3
2021-02-03 00:00:00,2021-02-03T00:00:00Z,3,2,1,2021,34,4
2021-02-04 00:00:00,2021-02-04T00:00:00Z,4,2,1,2021,35,5


In [0]:
spark.sql("drop table if exists default.dim_time")
dim_date.write.format("delta").mode("overwrite").saveAsTable("default.dim_date")